This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'api_key_here'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
from collections import defaultdict
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?collapse=daily&limit=1&api_key='+API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset': {'id': 10095370,
  'dataset_code': 'AFX_X',
  'database_code': 'FSE',
  'name': 'Carl Zeiss Meditec (AFX_X)',
  'description': 'Stock Prices for Carl Zeiss Meditec (2019-08-02) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704',
  'refreshed_at': '2019-08-16T22:30:11.817Z',
  'newest_available_date': '2019-08-16',
  'oldest_available_date': '2000-06-07',
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'frequency': 'daily',
  'type': 'Time Series',
  'premium': False,
  'limit': 1,
  'transform': None,
  'column_index': None,
  'start_date': '2000-06-07',
  'end_date': '2019-08-16',
  'data': [['2019-08-16',
    98.35,
    99.4,
    97.15,
    99.3,
    None,
    106762.0,
    10549874.0,
    None,
    None,
    None]],
  'collapse': 'daily',
  'order': None,
  'database_id': 6129}}

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

### 1. Collect data from FSE for the year 2017

We can pull data for the year 2017 by specifying the start and end dates. We will also use our API_KEY variable defined above.

In [5]:
d = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&api_key='+API_KEY)

### 2. Convert JSON object into a dictionary

We can convert the JSON object to a dictionary using the .json() method and verify this using an assert statement.

In [6]:
json_data = d.json()
assert(type(json_data) == dict)

#### Preparing for the following tasks

In order to accomplish the remaining questions, we will create some useful data structures and functions.

First, we can load the necessary pieces of information into its own dictionary (I used a defaultdict). I extracted the "column_names" and "data" fields from the original dictionary.

In [7]:
stock_data = defaultdict(list)
for i, k in enumerate(json_data['dataset']['column_names'][:11]):
    stock_data[k].append([entry[i] for entry in json_data['dataset']['data']])

#For some reason, our dictionary values are in a nested list instead of a single list, so let's take care of that:
for key in stock_data.keys():
    stock_data[key] = stock_data[key][0]

Next, we will define some helpful functions below:

In [8]:
def get_max(l):
    """
    Returns maximum value of list while ignoring None values
    """
    return max([i for i in l if i is not None])

def get_min(l):
    """
    Returns minimum value of list while ignoring None values
    """
    return min([i for i in l if i is not None])

def rounded(x):
    """
    Rounds input to two decimal places
    """
    return float("{:.2f}".format(x))

Now we can continue with the exercises.

### 3. Calculate the highest and lowest opening prices

In [9]:
print('The highest opening price is:', rounded(get_max(stock_data['Open'])))

print('The lowest opening price is:', rounded(get_min(stock_data['Open'])))

The highest opening price is: 53.11
The lowest opening price is: 34.0


### 4. Calculate the largest change in any one day

We will ignore any days where either the "High" or "Low" value is None.

In [10]:
daily_changes = []
for i, val in enumerate(stock_data['High']):
    if val is not None and stock_data['Low'][i] is not None:
        daily_changes.append(val - stock_data['Low'][i])
    else:
        continue

print('The largest daily change is:', rounded(max(daily_changes)))

The largest daily change is: 2.81


### 5. Calculate the largest change between any two days

Again, we will ignore entries with None values.

In [11]:
closing_price_changes = []
for i, val in enumerate(stock_data['Close']):
    if val is not None:
        for j, val2 in enumerate(stock_data['Close']):
            if val2 is not None:
                closing_price_changes.append(abs(val - val2))
            else:
                continue
    else:
        continue
print('The largest change between any two days is:', rounded(get_max(closing_price_changes)))

The largest change between any two days is: 19.03


### 6. Calculate the average daily trading volume

Thankfully, the "Traded Volume" field did not have any None values, so calculating the average is straightforward.

In [12]:
print('The average daily trading volume for 2017 is:', 
      rounded(sum(stock_data['Traded Volume']) / len(stock_data['Traded Volume'])))

The average daily trading volume for 2017 is: 89124.34


### 7. (Optional) Calculate the median daily trading volume

We will need to calculate the index of the median value by dividing the length of the "Traded Volume" list in half and rounding up to the nearest integer (i.e. taking the ceiling).

In [13]:
import math
median_index = int(math.ceil(len(stock_data['Traded Volume'])/2))
print('The median trading volume for 2017 is:', stock_data['Traded Volume'][median_index])

The median trading volume for 2017 is: 117165.0
